<a href="https://colab.research.google.com/github/runminghuang/Transfer_Learning_for_Stock_Prediction/blob/data-processing/Data_processing_signature_computing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import esig
import iisignature
import pandas as pd
import itertools
from sklearn.decomposition import PCA

## Data Import

In [2]:
x_train = pd.read_csv("/Users/runminghuang/Desktop/capstone/Data/x_train.csv")
y_train = pd.read_csv("/Users/runminghuang/Desktop/capstone/Data/y_train.csv")
x_test = pd.read_csv("/Users/runminghuang/Desktop/capstone/Data/x_test.csv")
y_test = pd.read_csv("/Users/runminghuang/Desktop/capstone/Data/test_rand.csv")

In [3]:
x_train

,ID,DATE,STOCK,INDUSTRY,INDUSTRY_GROUP,SECTOR,SUB_INDUSTRY,RET_1,VOLUME_1,RET_2,...,RET_16,VOLUME_16,RET_17,VOLUME_17,RET_18,VOLUME_18,RET_19,VOLUME_19,RET_20,VOLUME_20
0,0,0,2,18,5,3,44,-0.015748,0.147931,-0.015504,...,0.059459,0.630899,0.003254,-0.379412,0.008752,-0.110597,-0.012959,0.174521,-0.002155,-0.000937
1,1,0,3,43,15,6,104,0.003984,NaN,-0.090580,...,0.015413,NaN,0.003774,NaN,-0.018518,NaN,-0.028777,NaN,-0.034722,NaN
2,2,0,4,57,20,8,142,0.000440,-0.096282,-0.058896,...,0.008964,-0.010336,-0.017612,-0.354333,-0.006562,-0.519391,-0.012101,-0.356157,-0.006867,-0.308868
3,3,0,8,1,1,1,2,0.031298,-0.429540,0.007756,...,-0.031769,0.012105,0.033824,-0.290178,-0.001468,-0.663834,-0.013520,-0.562126,-0.036745,-0.631458
4,4,0,14,36,12,5,92,0.027273,-0.847155,-0.039302,...,-0.038461,-0.277083,-0.012659,0.139086,0.004237,-0.017547,0.004256,0.579510,-0.040817,0.802806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418590,418590,223,5703,32,10,4,77,0.021843,-0.217823,-0.021703,...,0.003160,-0.161543,0.007785,-0.141487,0.008205,-0.268215,-0.058448,0.057577,0.031174,-0.163350
418591,418591,223,5705,35,12,5,91,-0.006920,-0.375251,0.000000,...,0.007169,-0.955492,-0.016221,-0.171172,0.007819,1.540184,-0.015396,-0.502337,-0.011073,0.632885
418592,418592,223,5709,2,1,1,5,0.021869,-0.978856,-0.005929,...,0.001110,-0.476550,0.029714,-0.385972,-0.069148,1.780169,-0.082927,1.581453,0.098607,6.956960
418593,418593,223,5710,33,10,4,83,0.012248,-0.627169,0.010925,...,0.007450,-0.210079,0.023729,0.813948,0.026087,-0.868576,-0.051155,0.371052,0.041238,0.045695


In [4]:
# Load stock data
df = x_train.sort_values(by=['STOCK', 'DATE']).reset_index(drop=True)
df

,ID,DATE,STOCK,INDUSTRY,INDUSTRY_GROUP,SECTOR,SUB_INDUSTRY,RET_1,VOLUME_1,RET_2,...,RET_16,VOLUME_16,RET_17,VOLUME_17,RET_18,VOLUME_18,RET_19,VOLUME_19,RET_20,VOLUME_20
0,2377,1,0,37,12,5,94,-0.005967,0.136699,0.009031,...,-0.009219,-0.493354,-0.007660,-0.585497,-0.001063,-0.351363,0.005127,-0.324675,-0.019275,-0.291751
1,5198,4,0,37,12,5,94,0.001348,-0.269520,0.011100,...,-0.000762,-0.313575,0.007867,0.071338,0.007733,-0.405243,-0.003276,-0.424336,-0.010489,-0.050591
2,8017,5,0,37,12,5,94,-0.014405,0.192655,0.003614,...,0.002742,-0.367499,-0.005843,-0.405562,0.002930,-0.315935,0.010462,-0.474957,-0.003541,-0.260130
3,20826,11,0,37,12,5,94,0.008938,0.430916,0.002662,...,0.009097,0.023598,0.011266,0.079711,0.019038,-0.230167,-0.000287,-0.312123,0.008682,-0.226628
4,33843,21,0,37,12,5,94,-0.006523,-0.060371,-0.007632,...,-0.014461,-0.337686,-0.007224,-0.161117,-0.001461,-0.095494,0.012667,0.471895,-0.038752,1.532045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418590,391556,206,5716,50,17,7,114,0.029552,-0.075091,-0.001428,...,0.024553,0.076480,0.017026,0.170516,0.002276,-0.106224,-0.034597,0.123750,-0.015676,-0.228186
418591,394490,208,5716,50,17,7,114,0.008316,0.028099,-0.006688,...,0.020970,0.090287,0.002887,-0.050408,0.008736,-0.159294,0.027350,-0.022922,0.008186,-0.080569
418592,400150,210,5716,50,17,7,114,-0.004633,-0.173518,0.001687,...,-0.024517,0.695373,-0.015320,-0.149467,-0.035810,-0.262389,0.000896,-0.172450,0.008586,-0.482171
418593,403129,211,5716,50,17,7,114,0.010883,0.172313,0.008844,...,0.005832,-0.045512,-0.008823,-0.026153,-0.011428,-0.142636,0.011253,-0.224195,0.000609,-0.341878


plan：
1. NaN -> 0 or Mean of column of the stock
2. Feature Engineering: 40 rtn and vol dimension -> 10 rtn and vol dim (PCA or other means)  Or, use 1day 5day 20day rtn and vol
3. For each row, calculate the sig (dim=11, order 3)
4. Feature Engineering on the Sig outcome: 1464 dim -> 15 dim (Ridge regression/ Lasso)

## Data Preprocess

In [5]:
# Fill NaN with 0
df = df.fillna(0)
# Keep only the selected columns
# selected_columns = ['ID', 'DATE', 'STOCK'] + [col for col in df.columns if col.startswith(('RET_', 'VOLUME_'))]
# df = df[selected_columns]
df

,ID,DATE,STOCK,INDUSTRY,INDUSTRY_GROUP,SECTOR,SUB_INDUSTRY,RET_1,VOLUME_1,RET_2,...,RET_16,VOLUME_16,RET_17,VOLUME_17,RET_18,VOLUME_18,RET_19,VOLUME_19,RET_20,VOLUME_20
0,2377,1,0,37,12,5,94,-0.005967,0.136699,0.009031,...,-0.009219,-0.493354,-0.007660,-0.585497,-0.001063,-0.351363,0.005127,-0.324675,-0.019275,-0.291751
1,5198,4,0,37,12,5,94,0.001348,-0.269520,0.011100,...,-0.000762,-0.313575,0.007867,0.071338,0.007733,-0.405243,-0.003276,-0.424336,-0.010489,-0.050591
2,8017,5,0,37,12,5,94,-0.014405,0.192655,0.003614,...,0.002742,-0.367499,-0.005843,-0.405562,0.002930,-0.315935,0.010462,-0.474957,-0.003541,-0.260130
3,20826,11,0,37,12,5,94,0.008938,0.430916,0.002662,...,0.009097,0.023598,0.011266,0.079711,0.019038,-0.230167,-0.000287,-0.312123,0.008682,-0.226628
4,33843,21,0,37,12,5,94,-0.006523,-0.060371,-0.007632,...,-0.014461,-0.337686,-0.007224,-0.161117,-0.001461,-0.095494,0.012667,0.471895,-0.038752,1.532045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418590,391556,206,5716,50,17,7,114,0.029552,-0.075091,-0.001428,...,0.024553,0.076480,0.017026,0.170516,0.002276,-0.106224,-0.034597,0.123750,-0.015676,-0.228186
418591,394490,208,5716,50,17,7,114,0.008316,0.028099,-0.006688,...,0.020970,0.090287,0.002887,-0.050408,0.008736,-0.159294,0.027350,-0.022922,0.008186,-0.080569
418592,400150,210,5716,50,17,7,114,-0.004633,-0.173518,0.001687,...,-0.024517,0.695373,-0.015320,-0.149467,-0.035810,-0.262389,0.000896,-0.172450,0.008586,-0.482171
418593,403129,211,5716,50,17,7,114,0.010883,0.172313,0.008844,...,0.005832,-0.045512,-0.008823,-0.026153,-0.011428,-0.142636,0.011253,-0.224195,0.000609,-0.341878


## Feature Engineering Methods

### VIF (variance_inflation_factor)
VIF > 10：strong colinearity, considering remove this feature

VIF > 5：mild colinearity, need further research

VIF ≈ 1：feature independence is good

In [6]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
ret_cols = [col for col in df.columns if col.startswith('RET_')]
vol_cols = [col for col in df.columns if col.startswith('VOLUME_')]

features_for_vif = df[ret_cols + vol_cols]
vif_data = pd.DataFrame()
vif_data["feature"] = features_for_vif.columns
vif_data["VIF"] = [variance_inflation_factor(features_for_vif.values, i) for i in range(features_for_vif.shape[1])]
vif_data


,feature,VIF
0,RET_1,1.017835
1,RET_2,1.060485
2,RET_3,1.102149
3,RET_4,1.031332
4,RET_5,1.054145
5,RET_6,1.036464
6,RET_7,1.044032
7,RET_8,1.032669
8,RET_9,1.040597
9,RET_10,1.031541


## Pearson Correlation Matrix

In [6]:
import seaborn as sns
import matplotlib.pyplot as plt

corr_matrix = df[ret_cols + vol_cols].corr()
plt.figure(figsize=(20, 10))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Feature Correlation Heatmap")
plt.show()

NameError: name 'ret_cols' is not defined

Based on the VIF and Correlation matrix, we have several findings:

    1. VIF of VOLUME and RET data are close to 1, which means that COL_i (VOLUME_i) and COL_j(VOLUME_j) using VIF to explain the correlation correlation generally pass the test, basically will not reduce the independence and marginal contribution to the further model.
    1. RET data has less colinearity issue compared with VOLUME data
    2. The closer j to i, the higher correlation between VOLUME_i and VOLUME_j
    3. Normally, when |j-i| <= 5, the colinearity and correlation problem arise.

Therefore, we choose to use [RET_1, RET_5, RET_10, RET_15, RET_20], [VOLUME_1, VOLUME_5, VOLUME_10, VOLUME_15, VOLUME_20] as our simplified data.

In [7]:
other_cols = ['ID', 'DATE', 'STOCK', 'INDUSTRY', 'INDUSTRY_GROUP', 'SECTOR', 'SUB_INDUSTRY']
ret_cols = ['RET_1', 'RET_5', 'RET_10','RET_15','RET_20']
vol_cols = ['VOLUME_1', 'VOLUME_5', 'VOLUME_10','VOLUME_15','VOLUME_20']
df_featured = df[other_cols + ret_cols + vol_cols]
df_featured

,ID,DATE,STOCK,INDUSTRY,INDUSTRY_GROUP,SECTOR,SUB_INDUSTRY,RET_1,RET_5,RET_10,RET_15,RET_20,VOLUME_1,VOLUME_5,VOLUME_10,VOLUME_15,VOLUME_20
0,2377,1,0,37,12,5,94,-0.005967,0.012739,-0.006094,-0.015148,-0.019275,0.136699,0.746002,-0.268285,-0.382067,-0.291751
1,5198,4,0,37,12,5,94,0.001348,0.005654,-0.002856,-0.006286,-0.010489,-0.269520,-0.379779,-0.405866,-0.249143,-0.050591
2,8017,5,0,37,12,5,94,-0.014405,0.002700,0.007140,-0.002735,-0.003541,0.192655,0.001736,-0.229625,-0.209810,-0.260130
3,20826,11,0,37,12,5,94,0.008938,-0.002252,0.007473,0.005980,0.008682,0.430916,-0.130800,-0.326320,0.032272,-0.226628
4,33843,21,0,37,12,5,94,-0.006523,-0.006111,-0.009156,-0.000654,-0.038752,-0.060371,-0.503069,-0.471997,-0.510826,1.532045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418590,391556,206,5716,50,17,7,114,0.029552,-0.003393,-0.012152,-0.010348,-0.015676,-0.075091,-0.180967,0.009643,0.347596,-0.228186
418591,394490,208,5716,50,17,7,114,0.008316,-0.000402,-0.009604,-0.002417,0.008186,0.028099,-0.317379,-0.147916,0.626742,-0.080569
418592,400150,210,5716,50,17,7,114,-0.004633,0.000000,-0.010393,-0.009667,0.008586,-0.173518,-0.498952,-0.525759,-0.020833,-0.482171
418593,403129,211,5716,50,17,7,114,0.010883,0.012772,-0.025966,-0.009765,0.000609,0.172313,-0.357663,0.035364,-0.199628,-0.341878


## Signature Computing

In [8]:
# Define signature computation parameters
n = 3  # Signature order
features = ['DATE'] + ret_cols + vol_cols
d = len(features)
signature_data = []

# Sliding window parameters
W = 5   # Window size (number of time steps per sub-path)
S = 1    # Step size (how much we shift the window each time)

#### Test: Only select the first stock

In [9]:
# Select the first stock
first_stock_name = df_featured['STOCK'].unique()[0]  # Get the first stock name
first_stock_df = df_featured[df_featured['STOCK'] == first_stock_name].sort_values(by="DATE")  # Ensure chronological order

# Drop NaN values from selected features
first_stock_df = first_stock_df.dropna(subset=features)

# Ensure at least W rows exist before computing the signature
if len(first_stock_df) < W:
    raise ValueError(f"Not enough data for stock {first_stock_name} to compute signatures.")

# Convert to NumPy array for faster processing
path_data = first_stock_df[features].values  # Shape: (T, d)

# Precompute signature transformation for efficiency
sig_transform = iisignature.prepare(d, n)

# Prepare storage for signature data
signature_data = []

# Apply sliding window
for start in range(0, len(path_data) - W + 1, S):
    end = start + W
    window_path = path_data[start:end]  # Extract sub-path

    # Compute the signature for this window
    signature = iisignature.sig(window_path, n)

    # Store results with metadata
    signature_data.append({
        'Stock': first_stock_name,
        'Industry': first_stock_df.iloc[start]["INDUSTRY"].astype(int),
        'Industry_Group': first_stock_df.iloc[start]["INDUSTRY_GROUP"].astype(int),
        'Sub_Industry': first_stock_df.iloc[start]["SUB_INDUSTRY"].astype(int),
        'Sector': first_stock_df.iloc[start]["SECTOR"].astype(int),
        'Start Time': first_stock_df.iloc[start]["DATE"].astype(int),
        'End Time': first_stock_df.iloc[end - 1]["DATE"].astype(int),
        **{f'Sig_{i}': signature[i] for i in range(len(signature))}
    })

# Convert to DataFrame and display
signature_df = pd.DataFrame(signature_data)
signature_df

,Stock,Industry,Industry_Group,Sub_Industry,Sector,Start Time,End Time,Sig_0,Sig_1,Sig_2,...,Sig_1453,Sig_1454,Sig_1455,Sig_1456,Sig_1457,Sig_1458,Sig_1459,Sig_1460,Sig_1461,Sig_1462
0,0,37,12,94,5,1,21,20.0,-0.000556,-0.018850,...,-0.008964,-0.002326,-0.009416,-0.003678,-0.027091,-0.280932,-0.220834,-0.083236,-0.310192,1.011061
1,0,37,12,94,5,4,22,18.0,-0.006075,0.002594,...,-0.004898,0.003223,0.003468,-0.001965,-0.002002,-0.218490,-0.046288,0.018791,-0.105441,-0.014357
2,0,37,12,94,5,5,28,23.0,0.032978,-0.007860,...,-0.007420,0.004623,0.004590,-0.003163,-0.003760,-0.323834,-0.071490,0.026162,-0.162159,0.000320
3,0,37,12,94,5,11,32,21.0,0.010199,0.006678,...,-0.006948,0.004388,0.004181,-0.002922,-0.003611,-0.306219,-0.066714,0.027582,-0.152259,-0.001297
4,0,37,12,94,5,21,39,18.0,0.016542,0.012350,...,0.025069,0.006029,-0.056126,0.020099,0.031582,0.265190,0.493580,0.877977,0.007644,-1.134555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,0,37,12,94,5,181,206,25.0,0.018397,0.009228,...,0.004728,0.003484,-0.002943,0.004594,-0.002798,-0.143223,-0.048345,-0.118008,-0.188867,0.000085
62,0,37,12,94,5,188,208,20.0,0.007754,0.006620,...,0.002447,0.001903,-0.001690,0.002764,-0.001624,-0.079835,-0.029610,-0.072100,-0.109235,-0.000002
63,0,37,12,94,5,189,210,21.0,0.016720,-0.004582,...,0.001196,-0.000529,-0.001001,-0.008639,-0.000665,0.036519,-0.061381,-0.070871,0.001169,-0.130099
64,0,37,12,94,5,196,211,15.0,-0.001482,-0.013746,...,-0.000559,-0.000979,0.000583,0.000605,0.001237,-0.015496,-0.006629,0.028576,0.013845,-0.021200


#### Compute signatures for all stocks

In [ ]:
signature_data = []

# Process each stock separately
for stock_id, stock_df in df_featured.groupby("STOCK"):
    print(stock_id)
    stock_df = stock_df.sort_values(by="DATE")  # Ensure chronological order

    # Drop NaN values in selected features
    stock_df = stock_df.dropna(subset=features)

    # Ensure at least W rows exist before computing the signature
    if len(stock_df) < W:
        print(f"Skipping stock {stock_id}: Not enough data for sliding window computation.")
        continue

    # Convert to NumPy array for faster processing
    path_data = stock_df[features].values  # Shape: (T, d)

    # Precompute signature transformation for efficiency
    sig_transform = iisignature.prepare(d, n)

    # Apply sliding window
    for start in range(0, len(path_data) - W + 1, S):
        end = start + W
        window_path = path_data[start:end]  # Extract sub-path

        # Compute the signature for this window
        signature = iisignature.sig(window_path, n)

        # Store results with metadata
        signature_data.append({
            'Stock': stock_id,
            'Industry': stock_df.iloc[start]["INDUSTRY"].astype(int),
            'Industry_Group': stock_df.iloc[start]["INDUSTRY_GROUP"].astype(int),
            'Sub_Industry': stock_df.iloc[start]["SUB_INDUSTRY"].astype(int),
            'Sector': stock_df.iloc[start]["SECTOR"].astype(int),
            'Start Time': stock_df.iloc[start]["DATE"].astype(int),
            'End Time': stock_df.iloc[end - 1]["DATE"].astype(int),
        **{f'Sig_{i}': signature[i] for i in range(len(signature))}
    })


# Convert to DataFrame and display


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279


In [15]:
len(signature_data[0])

1470

In [11]:
signature_df = pd.DataFrame(signature_data)

KeyboardInterrupt: 

In [12]:
window_size = 5  # Define the sliding window size
signature_data = []  # Store signature results

for stock_id, stock_df in df_featured.groupby("STOCK"):
    stock_df = stock_df.sort_values(by="DATE")  # Ensure chronological order
    num_rows = stock_df.shape[0]

    # Apply sliding window
    for start in range(num_rows - window_size):  # Iterate with step size 1
        window_df = stock_df.iloc[start:start + window_size]  # Extract window slice
        date = stock_df.iloc[start + window_size]["DATE"]  # Capture the date that will use these sig for prediction

        path = window_df[features].values  # Shape (window_size, num_features)

        try:
            signature = iisignature.sig(path, n)  # Compute signature
            signature_list = list(signature)
        except ValueError as e:
            print(f"Skipping stock {stock_id}, on {date}, due to signature computation error: {e}")
            continue

        # Generate feature interaction labels
        multi_index_labels = ["Zeroth Level"]
        level_counts = [0]
        d = len(features)  # Number of features

        for level in range(1, n + 1):
            indices = list(itertools.product(range(d), repeat=level))  # Generate index sequences
            for idx in indices:
                term_label = '-'.join([features[i] for i in idx])  # Convert index to feature names
                multi_index_labels.append(term_label)
                level_counts.append(level)

        # Ensure array sizes match
        min_length = min(len(signature_list), len(multi_index_labels), len(level_counts))
        signature_list = signature_list[:min_length]
        multi_index_labels = multi_index_labels[:min_length]
        level_counts = level_counts[:min_length]

        # Store results with stock metadata
        for i in range(min_length):
            signature_data.append({
                'Stock': stock_id,
                'Date': date,  # Capture the date that will use these sig for prediction
                'Signature Term': signature_list[i],
                'Order Level': level_counts[i],
                'Feature Interaction': multi_index_labels[i]
            })
    break  # 仅调试一个股票
signature_data
# Convert to DataFrame and display
# signature_df = pd.DataFrame(signature_data)

[{'Stock': 0,
  'Date': np.float64(22.0),
  'Signature Term': np.float64(20.0),
  'Order Level': 0,
  'Feature Interaction': 'Zeroth Level'},
 {'Stock': 0,
  'Date': np.float64(22.0),
  'Signature Term': np.float64(-0.0005558124262107032),
  'Order Level': 1,
  'Feature Interaction': 'DATE'},
 {'Stock': 0,
  'Date': np.float64(22.0),
  'Signature Term': np.float64(-0.0188498011515871),
  'Order Level': 1,
  'Feature Interaction': 'RET_1'},
 {'Stock': 0,
  'Date': np.float64(22.0),
  'Signature Term': np.float64(-0.003062056392436098),
  'Order Level': 1,
  'Feature Interaction': 'RET_5'},
 {'Stock': 0,
  'Date': np.float64(22.0),
  'Signature Term': np.float64(0.0144939589800323),
  'Order Level': 1,
  'Feature Interaction': 'RET_10'},
 {'Stock': 0,
  'Date': np.float64(22.0),
  'Signature Term': np.float64(-0.019477489574318097),
  'Order Level': 1,
  'Feature Interaction': 'RET_15'},
 {'Stock': 0,
  'Date': np.float64(22.0),
  'Signature Term': np.float64(-0.19706974539994704),
  'Or

In [ ]:
signature_df

""
